In [1]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### Part 1. Define simple computational graph

In [2]:
# define variable in tensorflow
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

# define computational graph
addition = tf.add(X,Y, name="addition")

### Create the session

In [3]:
with tf.Session() as session:
    ## run the session and feed the variable into placeholder
    result = session.run(addition, feed_dict={X: [1], Y: [4]})
    print(result)

[5.]


### Options for loading Data

In [4]:
import pandas as pd

In [5]:
## Option.1: Preload data into memory - quick and easy 

In [6]:
## Option.2: Feed data Step-by-Step

In [7]:
## Option.3: Set Up a Data pipeline -> used when dataset is large scale dataset

### Excercise 1. whole pipeline for training a model

#### Step1. Load train-test dataset

In [8]:
## training
training_data_df = pd.read_csv("../sale-dataset/sales_data_training.csv", dtype=float)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']]

## test
test_data_df = pd.read_csv("../sale-dataset/sales_data_test.csv", dtype=float)
X_testing = test_data_df.drop("total_earnings", axis=1).values
Y_testing = test_data_df[["total_earnings"]]

#### Step2. normalize dataset

In [9]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler(feature_range=(0,1))
Y_scaler = MinMaxScaler(feature_range=(0,1))

X_scaled_training = X_scaler.fit_transform(X_training)
X_scaled_testing = X_scaler.transform(X_testing)

Y_scaled_training = X_scaler.fit_transform(Y_training)
Y_scaled_testing = X_scaler.transform(Y_testing)

In [10]:
print(X_scaled_training.shape)
print(X_scaled_testing.shape)

(1000, 9)
(400, 9)


#### Step3. Build a Neural Network
In this part, we build a simple three layers network
- Input: 9 values
- Output: 1 node - regression prediction

In [11]:
## define model parameters
learning_rate = 0.001
training_epochs = 100
display_step = 5

## define how many inputs and outputs are in our neural network
number_of_input = 9
number_of_output = 1

## define layer nodes
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

Secion 1. Define the layers of the neural network itself

In [14]:
tf.reset_default_graph()

## Input layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_input))
    
## Layer 1
with tf.variable_scope("Layer1"):
    weights = tf.get_variable(name="weights1", shape=[number_of_input, layer_1_nodes],
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], 
                            initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X,weights) + biases)
    
    
## Layer 2
with tf.variable_scope("Layer2"):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes],
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], 
                            initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output,weights) + biases)
    
    
## Layer 3
with tf.variable_scope("Layer3"):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes],
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], 
                            initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output,weights) + biases)

## Output
with tf.variable_scope("output"):
    weights = tf.get_variable(name="weights3", shape=[layer_3_nodes, number_of_output],
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[number_of_output], 
                            initializer=tf.zeros_initializer())
    prediction = tf.nn.relu(tf.matmul(layer_3_output,weights) + biases)


Section 2. Define the cost function of the neural network

In [15]:
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

Section 3. Define optimizer function

In [16]:
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Section 4. Tensorboard

In [ ]:
with tf.variable_scope("logging"):
    tf.summary.scaler('current_cost', cost)
    summary = tf.summary.merge_all()

#### Step 4. Training loop

In [21]:
## open the session
with tf.Session() as session:
    ## run global parameter to initialize all layer variable
    session.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
        
        ## save the training log
        training_writer = tf.summary.FileWriter('./logs/training', session.graph)
        testing_writer = tf.summary.FileWriter('./logs/testing', session.graph)

        if epoch % 10 == 0:
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y: Y_scaled_training})
            testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
            print("Train Cost: {0}%, Test Cost: {1}%".format(training_cost*100, testing_cost*100))
            
            ## write current summary into log
            training_writer.add_summary(training_summary, epoch)
            test

    print("Training Complete")
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
    print("Final Train Cost: {0}%, Final Test Cost: {1}%".format(final_training_cost*100, final_testing_cost*100))

Train Cost: 9.673036634922028%, Test Cost: 10.676956176757812%
Train Cost: 2.595405653119087%, Test Cost: 2.5708818808197975%
Train Cost: 1.203997153788805%, Test Cost: 1.3577800244092941%
Train Cost: 0.636826828122139%, Test Cost: 0.6675867829471827%
Train Cost: 0.2775292843580246%, Test Cost: 0.2935332711786032%
Train Cost: 0.1570355030708015%, Test Cost: 0.15946039929986%
Train Cost: 0.09355543297715485%, Test Cost: 0.08521698182448745%
Train Cost: 0.062116922345012426%, Test Cost: 0.05524709704332054%
Train Cost: 0.041851497371681035%, Test Cost: 0.040144030936062336%
Train Cost: 0.03000313590746373%, Test Cost: 0.03089548263233155%
Training Complete
Final Train Cost: 0.023098019300960004%, Final Test Cost: 0.025276138330809772%


#### Step 5. visualize training log in Tensorboard -> 0.0.0.0:6006

In [27]:
! tensorboard --logdir=./logs

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa